# # Project: UFC Fight Data Processing

#### This file processes UFC fight data by:

- Performing an initial analysis of multiple datasets.
- Selecting and merging key datasets.
- Cleaning the data: removing duplicates, handling missing values, and standardizing columns.
- Producing a final dataset containing detailed information about events and fighters.


### # Load Datasets
**Description**:  
Load all datasets containing information about events and fighters.

In [63]:
import pandas as pd

In [64]:
# Load all datasets
fighter_stats = pd.read_csv('../data/raw/Fighter_stats/fighter_stats.csv')
large_dataset = pd.read_csv('../data/raw/Large_set/large_dataset.csv')
medium_dataset = pd.read_csv('../data/raw/Medium_set/medium_dataset.csv')
ufc = pd.read_csv('../data/raw/ufc.csv')
ufc_master = pd.read_csv('../data/raw/ufc-master.csv')
completed_events_small = pd.read_csv('../data/raw/Small_set/completed_events_small.csv')

### # Display Columns for All Datasets
 **Description**:  
Display the column names of all datasets to understand their structure.

In [65]:
# Function to print dataset columns
def print_dataset_columns(df, df_name):
    print(f"\n=== Columns in {df_name} ===")
    print(df.columns.tolist())
    print(f"Total Columns: {len(df.columns)}")

# Display columns for all datasets
print_dataset_columns(fighter_stats, "Fighter Stats")
print_dataset_columns(large_dataset, "Large Dataset")
print_dataset_columns(medium_dataset, "Medium Dataset")
print_dataset_columns(ufc, "UFC Dataset")
print_dataset_columns(ufc_master, "UFC Master Dataset")
print_dataset_columns(completed_events_small, "Completed Events Small Dataset")

# Explanation of chosen datasets
print("\nFrom the above datasets, we selected:")
print("- Large Dataset: for detailed fight information, including event names and fighters.")
print("- Medium Dataset: for additional event metadata, such as dates and locations.")


=== Columns in Fighter Stats ===
['name', 'wins', 'losses', 'height', 'weight', 'reach', 'stance', 'age', 'SLpM', 'sig_str_acc', 'SApM', 'str_def', 'td_avg', 'td_acc', 'td_def', 'sub_avg']
Total Columns: 16

=== Columns in Large Dataset ===
['event_name', 'r_fighter', 'b_fighter', 'winner', 'weight_class', 'is_title_bout', 'gender', 'method', 'finish_round', 'total_rounds', 'time_sec', 'referee', 'r_kd', 'r_sig_str', 'r_sig_str_att', 'r_sig_str_acc', 'r_str', 'r_str_att', 'r_str_acc', 'r_td', 'r_td_att', 'r_td_acc', 'r_sub_att', 'r_rev', 'r_ctrl_sec', 'r_wins_total', 'r_losses_total', 'r_age', 'r_height', 'r_weight', 'r_reach', 'r_stance', 'r_SLpM_total', 'r_SApM_total', 'r_sig_str_acc_total', 'r_td_acc_total', 'r_str_def_total', 'r_td_def_total', 'r_sub_avg', 'r_td_avg', 'b_kd', 'b_sig_str', 'b_sig_str_att', 'b_sig_str_acc', 'b_str', 'b_str_att', 'b_str_acc', 'b_td', 'b_td_att', 'b_td_acc', 'b_sub_att', 'b_rev', 'b_ctrl_sec', 'b_wins_total', 'b_losses_total', 'b_age', 'b_height', 'b_

### # Preprocess Selected Datasets
 **Description**:  
 Preprocess the selected datasets (`large_dataset` and `medium_dataset`):
 - Rename columns for consistency.
 - Standardize values in `event_name`.
 - Remove duplicates.

In [66]:
# Rename columns for consistency
medium_dataset = medium_dataset.rename(columns={'event': 'event_name'})

# Strip extra spaces and standardize to lowercase in 'event_name'
large_dataset['event_name'] = large_dataset['event_name'].str.strip().str.lower()
medium_dataset['event_name'] = medium_dataset['event_name'].str.strip().str.lower()

# Check for duplicates based on 'event_name', 'r_fighter', and 'b_fighter'
large_duplicates = large_dataset.duplicated(subset=['event_name', 'r_fighter', 'b_fighter'])
medium_duplicates = medium_dataset.duplicated(subset=['event_name', 'date', 'location'])

# Print the number of duplicates
print("Duplicates in Large Dataset (based on 'event_name', 'r_fighter', 'b_fighter'):", large_duplicates.sum())
print("Duplicates in Medium Dataset (based on 'event_name', 'date', 'location'):", medium_duplicates.sum())

# Drop duplicates based on the relevant columns
large_dataset = large_dataset.drop_duplicates(subset=['event_name', 'r_fighter', 'b_fighter'])
medium_dataset = medium_dataset.drop_duplicates(subset=['event_name', 'date', 'location'])


Duplicates in Large Dataset (based on 'event_name', 'r_fighter', 'b_fighter'): 0
Duplicates in Medium Dataset (based on 'event_name', 'date', 'location'): 6898


In [67]:
duplicate_dates = medium_dataset[medium_dataset.duplicated(subset=['date'], keep=False)]
print(duplicate_dates)


                                    event_name        date  \
3596       ufc fight night: bader vs. nogueira  11/19/2016   
3607       ufc fight night: mousasi vs. hall 2  11/19/2016   
4581  ufc fight night: macdonald vs saffiedine   10/4/2014   
4592          ufc fight night: nelson vs story   10/4/2014   
4654   ufc fight night: henderson vs dos anjos   8/23/2014   
4665            ufc fight night: bisping vs le   8/23/2014   
4736      ufc fight night: swanson vs stephens   6/28/2014   
4747     ufc fight night: te huna vs marquardt   6/28/2014   
4779      ufc fight night: miocic vs maldonado   5/31/2014   
4791         ufc fight night: munoz vs mousasi   5/31/2014   

                                       location         r_fighter  \
3596               Sao Paulo, Sao Paulo, Brazil        Ryan Bader   
3607  Belfast, Northern Ireland, United Kingdom    Gegard Mousasi   
4581               Halifax, Nova Scotia, Canada    Rory MacDonald   
4592                          Stockholm, 


### # Merge Datasets
**Description**:  
Merge `large_dataset` and `medium_dataset` on the `event_name` column.

In [68]:
# Merge the datasets on 'event_name', excluding 'method_details'
merged_dataset = pd.merge(
    large_dataset, 
    medium_dataset[['event_name', 'date', 'location']], 
    on='event_name', 
    how='left'
)

In [69]:
# Get the current column order
columns_order = merged_dataset.columns.tolist()
event_name_idx = columns_order.index('event_name')

# Define the new column order
new_columns_order = (
    columns_order[:event_name_idx + 1] +  # Include 'event_name' and everything before it
    ['date', 'location'] +               # Add 'date' and 'location' after 'event_name'
    [col for col in columns_order if col not in ['date', 'location', 'event_name']]  # Remaining columns
)

# Apply the new column order using pd.Index for deduplication
merged_dataset = merged_dataset.loc[:, pd.Index(new_columns_order).drop_duplicates()]


### # Data Quality Checks
**Description**:  
 Perform quality checks on the merged dataset:
 - Check for missing values.
 - Identify duplicate rows.
 - Display data types and unique values.

In [70]:
# Function to perform data quality checks
def data_quality_checks(df, df_name):
    print(f"\n=== Data Quality Checks for {df_name} ===")

    # Missing values
    missing_values = df.isnull().sum()
    print("\nMissing Values:")
    print(missing_values[missing_values > 0])

    # Duplicate rows
    duplicate_rows = df.duplicated().sum()
    print(f"\nNumber of Duplicate Rows: {duplicate_rows}")

    # Data types
    print("\nData Types:")
    print(df.dtypes)

    # Unique values
    print("\nUnique Values Count:")
    print(df.nunique())

    # Statistical summary for numerical columns
    print("\nStatistical Summary for Numerical Columns:")
    print(df.describe())


# Perform quality checks on the merged dataset
data_quality_checks(merged_dataset, "Merged Dataset")


=== Data Quality Checks for Merged Dataset ===

Missing Values:
date              12
location          12
total_rounds      31
referee           32
r_age             76
r_reach          412
r_stance          26
b_age            190
b_reach          888
b_stance          68
age_diff         213
reach_diff      1038
dtype: int64

Number of Duplicate Rows: 0

Data Types:
event_name             object
date                   object
location               object
r_fighter              object
b_fighter              object
                       ...   
td_acc_total_diff     float64
str_def_total_diff    float64
td_def_total_diff     float64
sub_avg_diff          float64
td_avg_diff           float64
Length: 97, dtype: object

Unique Values Count:
event_name             683
date                   677
location               168
r_fighter             1796
b_fighter             2301
                      ... 
td_acc_total_diff      291
str_def_total_diff     187
td_def_total_diff      313
sub_avg

### # Block 6: Fill Missing Values
**Description**:  
 Handle missing values in the merged dataset:
 - For string columns, fill with the most frequent value (mode).
 - For numeric columns, fill with the median.


In [71]:
# Fill missing values for string and numeric columns
def fill_missing_values(df):
    for col in df.columns:
        if df[col].dtype == 'object':  # For string columns
            mode_value = df[col].mode()[0]  # Most frequent value
            df[col] = df[col].fillna(mode_value)
        elif pd.api.types.is_numeric_dtype(df[col]):  # For numeric columns
            df[col] = df[col].fillna(df[col].median())  # Median for numeric columns
    return df

# Apply the function to the merged dataset
merged_dataset = fill_missing_values(merged_dataset)

# Check for missing values after filling
print("\nMissing values after filling:")
print(merged_dataset.isnull().sum())


Missing values after filling:
event_name            0
date                  0
location              0
r_fighter             0
b_fighter             0
                     ..
td_acc_total_diff     0
str_def_total_diff    0
td_def_total_diff     0
sub_avg_diff          0
td_avg_diff           0
Length: 97, dtype: int64


In [74]:
# Function to print dataset shape and column names
def print_dataset_info(df, df_name):
    print(f"\n=== Dataset: {df_name} ===")
    print(f"Number of Rows: {df.shape[0]}")
    print(f"Number of Columns: {df.shape[1]}")
    print("Columns:", df.columns.tolist())

# Print info for each dataset
print_dataset_info(large_dataset, "Large Dataset")
print_dataset_info(medium_dataset, "Medium Dataset")

# If the merged dataset exists
if 'merged_dataset' in locals():
    print_dataset_info(merged_dataset, "Merged Dataset")



=== Dataset: Large Dataset ===
Number of Rows: 7439
Number of Columns: 95
Columns: ['event_name', 'r_fighter', 'b_fighter', 'winner', 'weight_class', 'is_title_bout', 'gender', 'method', 'finish_round', 'total_rounds', 'time_sec', 'referee', 'r_kd', 'r_sig_str', 'r_sig_str_att', 'r_sig_str_acc', 'r_str', 'r_str_att', 'r_str_acc', 'r_td', 'r_td_att', 'r_td_acc', 'r_sub_att', 'r_rev', 'r_ctrl_sec', 'r_wins_total', 'r_losses_total', 'r_age', 'r_height', 'r_weight', 'r_reach', 'r_stance', 'r_SLpM_total', 'r_SApM_total', 'r_sig_str_acc_total', 'r_td_acc_total', 'r_str_def_total', 'r_td_def_total', 'r_sub_avg', 'r_td_avg', 'b_kd', 'b_sig_str', 'b_sig_str_att', 'b_sig_str_acc', 'b_str', 'b_str_att', 'b_str_acc', 'b_td', 'b_td_att', 'b_td_acc', 'b_sub_att', 'b_rev', 'b_ctrl_sec', 'b_wins_total', 'b_losses_total', 'b_age', 'b_height', 'b_weight', 'b_reach', 'b_stance', 'b_SLpM_total', 'b_SApM_total', 'b_sig_str_acc_total', 'b_td_acc_total', 'b_str_def_total', 'b_td_def_total', 'b_sub_avg', 'b_

### Save the Final Dataset
 **Description**:  
 Save the cleaned and processed dataset to a file.

In [73]:
# Save the merged dataset to a file
output_path = '../data/processed/ufc_data_final.csv'
merged_dataset.to_csv(output_path, index=False)
print(f"\nMerged dataset saved successfully at {output_path}")


Merged dataset saved successfully at ../data/processed/ufc_data_final.csv
